# Import packages and install dependencies

In [2]:
! git clone https://github.com/GryffindorLi/CS4240_Deep_Learning_Reproduce.git
! pip install SentencePiece
! pip install simpletransformers

Cloning into 'CS4240_Deep_Learning_Reproduce'...
remote: Enumerating objects: 403, done.
remote: Counting objects: 100% (403/403), done.
remote: Compressing objects: 100% (312/312), done.
remote: Total 403 (delta 267), reused 163 (delta 88), pack-reused 0
Receiving objects: 100% (403/403), 1.65 MiB | 5.54 MiB/s, done.
Resolving deltas: 100% (267/267), done.
     |████████████████████████████████| 1.2 MB 34.1 MB/s 
     |████████████████████████████████| 249 kB 34.2 MB/s 
     |████████████████████████████████| 1.8 MB 64.9 MB/s 
     |████████████████████████████████| 4.0 MB 53.2 MB/s 
     |████████████████████████████████| 325 kB 50.8 MB/s 
     |████████████████████████████████| 6.6 MB 60.2 MB/s 
     |████████████████████████████████| 10.1 MB 54.0 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 895 kB 62.4 MB/s 
     |████████████████████████████████| 77 kB 8.9 MB/s 
     

In [1]:
import pandas as pd
import sklearn.metrics as skm
import os
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from itertools import combinations

# Data loader and multi-label generation

In [2]:
df = pd.read_csv("CS4240_Deep_Learning_Reproduce/data/labeled.csv")
cols = df.columns[2:]

labels = []
for comb in combinations(cols, 2):
  if len(comb) > 1:
      labels.append( comb[0] + " and " + comb[1])
  else:
      labels.append(comb[0])

for i in cols: 
  labels.append(i)

print(len(labels),labels)

def data_loader_dl(datapath):
  df = pd.read_csv(datapath)
  arr = []
  cols = df.columns
  for index,row in df.iterrows():
    temp = []
    count = 0
    for index in range(len(row)):
      if row[index] == 1 and len(temp) < 2:
        temp.append(cols[index])
      if len(temp) == 2:
        break
    if len(temp) == 2 :
      label = temp[0] + " and " + temp[1]
      arr.append(label)
    else :
      arr.append(temp[0])
      
  df["labels"] = arr

  return df[['text','labels']]


train_set = data_loader_dl("CS4240_Deep_Learning_Reproduce/data/labeled.csv")
test_set = data_loader_dl("CS4240_Deep_Learning_Reproduce/data/test.csv")

66 ['fairness and non-moral', 'fairness and purity', 'fairness and degradation', 'fairness and loyalty', 'fairness and care', 'fairness and cheating', 'fairness and betrayal', 'fairness and subversion', 'fairness and authority', 'fairness and harm', 'non-moral and purity', 'non-moral and degradation', 'non-moral and loyalty', 'non-moral and care', 'non-moral and cheating', 'non-moral and betrayal', 'non-moral and subversion', 'non-moral and authority', 'non-moral and harm', 'purity and degradation', 'purity and loyalty', 'purity and care', 'purity and cheating', 'purity and betrayal', 'purity and subversion', 'purity and authority', 'purity and harm', 'degradation and loyalty', 'degradation and care', 'degradation and cheating', 'degradation and betrayal', 'degradation and subversion', 'degradation and authority', 'degradation and harm', 'loyalty and care', 'loyalty and cheating', 'loyalty and betrayal', 'loyalty and subversion', 'loyalty and authority', 'loyalty and harm', 'care and c

# Model

In [3]:
! rm -rf outputs\
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# specify hyperparameters and model arguments
model_args = ClassificationArgs(output_dir= "outputs/untrained_model")
model_args.overwrite_output_dir = True
model_args.use_multiprocessing = True
model_args.train_batch_size = 32
model_args.eval_batch_size = 32
model_args.labels_list = labels

# init bert
model = ClassificationModel('bert', 'bert-base-cased', num_labels=len(labels), args=model_args, use_cuda=True)
# Evaluate the untrained model on test set
result, _ , wrong_predictions = model.eval_model(test_set)
print(1 - (len(wrong_predictions)/len(test_set)))

for i in [3]:
  model_args.num_train_epochs = i
  model_args.output_dir = "outputs/epochs_"+str(i)

  # Create a TransformerModel
  model = ClassificationModel('bert', 'bert-base-cased', num_labels=len(labels), args=model_args, use_cuda=True)

  # Train the model
  model.train_model(train_set, show_running_loss=True)

  # Evaluate the model on test set
  result, _ , wrong_predictions = model.eval_model(test_set)
  print(1 - (len(wrong_predictions)/len(test_set)))
  source = '/content/outputs/epochs_'+str(i)+'/eval_results.txt'
  dst = '/content/outputs/epochs_'+str(i)+'/eval_results_epochs_'+str(i)+'.txt'

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

...

Running Epoch 1 of 3:   0%|          | 0/175 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/175 [00:00<?, ?it/s]

  0%|          | 0/3492 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/110 [00:00<?, ?it/s]

0.6331615120274914
